In [1]:
import pandas as pd

In [2]:
diagnoses = pd.read_csv('/Users/grace/mimic/csv/DIAGNOSES_ICD.csv', sep=',')
# diagnoses.head()

### Filter data by sepsis diagnosis

In [3]:
target_diag  = diagnoses[diagnoses.ICD9_CODE.isin(['77181', '99591', '99592', '67020', '67022', '67024'])]

In [4]:
target_diag.shape

(5409, 5)

### Total number of patients

In [5]:
target_diag.count()

ROW_ID        5409
SUBJECT_ID    5409
HADM_ID       5409
SEQ_NUM       5409
ICD9_CODE     5409
dtype: int64

In [6]:
len(target_diag.SUBJECT_ID.unique())

4781

In [7]:
target_diag.groupby(target_diag.SUBJECT_ID)#['ICD9_CODE'].count()#.sort('ICD9_CODE').index[-1]

In [8]:
target_diag.groupby(target_diag.SUBJECT_ID).count()[['ICD9_CODE']].sort_values(by=['ICD9_CODE'], ascending=False)[:10]

,ICD9_CODE
SUBJECT_ID,
41976,12
4787,8
3977,7
3866,7
73713,7
29426,6
49623,6
19029,5
9555,5


In [9]:
labs = pd.read_csv('/Users/grace/mimic/csv/LABEVENTS.csv', sep=',')
# labs.head()

In [10]:
labs[labs.ITEMID==51009]

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
22,303,3,NaN,51009,2101-10-13 03:00:00,16.8,16.8,ug/mL,abnormal
49,330,3,NaN,51009,2101-10-14 03:00:00,17.2,17.2,ug/mL,abnormal
127,408,3,NaN,51009,2101-10-16 04:00:00,21.1,21.1,ug/mL,abnormal
139,420,3,NaN,51009,2101-10-18 05:25:00,16.8,16.8,ug/mL,abnormal
140,421,3,NaN,51009,2101-10-18 09:30:00,41.2,41.2,ug/mL,abnormal
739,145,3,NaN,51009,2101-10-11 19:10:00,25.7,25.7,ug/mL,abnormal
1073,2493,4,NaN,51009,2191-05-18 08:05:00,12.6,12.6,ug/mL,abnormal
1168,2588,4,NaN,51009,2191-05-21 05:37:00,18.2,18.2,ug/mL,abnormal
1179,2599,4,NaN,51009,2191-05-21 16:35:00,16.5,16.5,ug/mL,abnormal
1208,2628,4,NaN,51009,2191-05-22 19:20:00,17.6,17.6,ug/mL,abnormal


In [11]:
lab_items = pd.read_csv('/Users/grace/mimic/csv/D_LABITEMS.csv', sep=',')
# lab_items.head()

In [12]:
lab_items[lab_items.LABEL.str.find('Vancomycin')>=0]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
335,209,51009,Vancomycin,Blood,Chemistry,20578-1


In [13]:
va_labs = labs[(labs.ITEMID==51009) & (labs.SUBJECT_ID.isin(target_diag.SUBJECT_ID.unique()))]
va_labs.groupby(va_labs.SUBJECT_ID).count()[['CHARTTIME']].sort_values(by=['CHARTTIME'], ascending=False)[:10]

,CHARTTIME
SUBJECT_ID,
13033,138
18353,114
25030,110
19620,104
433,98
11923,97
22815,84
20124,81
16655,76


In [14]:
target_diag[target_diag.SUBJECT_ID==25030]

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
279041,279660,25030,102205,10.0,99592
279050,279669,25030,128473,8.0,99592
279058,279677,25030,132107,3.0,99592


### Patient Subject_ID = 25030

In [15]:
#diagnosis
diagnoses[diagnoses.SUBJECT_ID==25030].shape

(53, 5)

In [16]:
unique_d_for_25030 = diagnoses[diagnoses.SUBJECT_ID==25030]['ICD9_CODE'].unique()
# unique_d_for_25030

In [17]:
dia_items = pd.read_csv('/Users/grace/mimic/csv/D_ICD_DIAGNOSES.csv', sep=',')
# dia_items.head()

In [18]:
dia_items[dia_items.ICD9_CODE=='7070']

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE


In [21]:
sep_diag = dia_items[dia_items.ICD9_CODE.isin(unique_d_for_25030)][['ICD9_CODE', 'SHORT_TITLE']].set_index('ICD9_CODE').to_dict()
sep_diag = sep_diag['SHORT_TITLE']
sep_diag['7070'] = ''
# sep_diag

In [24]:
# dia_all_25030['SHORT_TITLE'] = dia_all_25030['ICD9_CODE'].apply(lambda x: sep_diag[x])
# dia_all_25030
# dia_all_25030.ICD9_CODE.unique()

## lab

In [25]:
unique_lab_for_25030 = labs[labs.SUBJECT_ID==25030]['ITEMID'].unique()
# unique_lab_for_25030

In [26]:
lab_items[lab_items.ITEMID.isin(unique_lab_for_25030)]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
5,551,51351,Lymphs,Cerebrospinal Fluid (CSF),Hematology,26479-6
6,552,51352,Macrophage,Cerebrospinal Fluid (CSF),Hematology,30426-1
9,555,51355,Monocytes,Cerebrospinal Fluid (CSF),Hematology,26486-1
14,560,51360,Polys,Cerebrospinal Fluid (CSF),Hematology,26517-3
16,562,51362,"RBC, CSF",Cerebrospinal Fluid (CSF),Hematology,26454-9
17,563,51363,"WBC, CSF",Cerebrospinal Fluid (CSF),Hematology,26465-5
40,586,51386,Bands,Other Body Fluid,Hematology,26510-8
73,619,51419,Eosinophils,Other Body Fluid,Hematology,26452-3
78,624,51424,Immunophenotyping,Other Body Fluid,Hematology,NaN


In [27]:
labels = lab_items[lab_items.ITEMID.isin(unique_lab_for_25030)][['LABEL', 'ITEMID']].set_index('ITEMID').to_dict()
labels = labels['LABEL']
# labels

In [28]:
lab_all_25030 = labs[labs.SUBJECT_ID==25030]
lab_all_25030.shape

(6828, 9)

In [29]:
lab_all_25030['LABEL'] = lab_all_25030.ITEMID.apply(lambda x: labels[x])

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## procedure

In [30]:
proc = pd.read_csv('/Users/grace/mimic/csv/PROCEDURES_ICD.csv', sep=',')
# proc.head()

In [31]:
unique_proc_for_25030 = proc[proc.SUBJECT_ID==25030]['ICD9_CODE'].unique()
# unique_proc_for_25030

In [32]:
proc_items = pd.read_csv('/Users/grace/mimic/csv/D_ICD_PROCEDURES.csv', sep=',')
# proc_items.head()

In [33]:
proc_items[proc_items.ICD9_CODE.isin(unique_proc_for_25030)]

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
336,148,17,Infusion of vasopressor,Infusion of vasopressor agent
340,152,22,IVUS intrathoracic ves,Intravascular imaging of intrathoracic vessels
375,187,331,Spinal tap,Spinal tap
421,233,234,Ventricl shunt-abdomen,Ventricular shunt to abdominal cavity and organs
453,552,311,Temporary tracheostomy,Temporary tracheostomy
505,604,331,Incision of lung,Incision of lung
702,801,3893,Venous cath NEC,"Venous catheterization, not elsewhere classified"
704,803,3895,Ven cath renal dialysis,Venous catheterization for renal dialysis
1586,1291,5498,Peritoneal dialysis,Peritoneal dialysis
1660,1365,4432,Percu gastrojejunostomy,Percutaneous [endoscopic] gastrojejunostomy


In [34]:
short_titles = proc_items[proc_items.ICD9_CODE.isin(unique_proc_for_25030)][['ICD9_CODE', 'SHORT_TITLE']].set_index('ICD9_CODE').to_dict()
short_titles = short_titles['SHORT_TITLE']
# short_titles

In [35]:
proc_all_25030 = proc[proc.SUBJECT_ID==25030]
proc_all_25030.shape

(31, 5)

In [36]:
proc_all_25030['SHORT_TITLE'] = proc[proc.SUBJECT_ID==25030].ICD9_CODE.apply(lambda x: short_titles[x])
proc_all_25030.to_csv('./proc.csv')

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### Patient General Information

In [37]:
admission = pd.read_csv('/Users/grace/mimic/csv/ADMISSIONS.csv', sep=',')
# admission.head()

In [38]:
admission[admission.SUBJECT_ID==25030]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
30134,30576,25030,168158,2113-09-13 21:01:00,2113-09-21 18:23:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2113-09-13 09:10:00,2113-09-13 20:45:00,HYPERTENSIVE URGENCY,0,1
30135,30577,25030,172599,2116-05-21 20:56:00,2116-07-23 17:05:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2116-05-21 15:05:00,2116-05-21 23:38:00,INTERCRANIAL HEMORRHAGE,0,1
30136,30578,25030,132107,2117-01-09 21:25:00,2117-01-15 12:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2117-01-09 17:54:00,2117-01-09 21:30:00,SEPSIS,0,1
30137,30579,25030,128473,2117-07-16 17:46:00,2117-07-30 14:52:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2117-07-16 09:42:00,2117-07-16 19:10:00,FEVER-LINE SEPSIS,0,1
30138,30580,25030,102205,2117-09-05 11:03:00,2117-09-05 12:00:00,2117-09-05 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2117-09-05 09:13:00,2117-09-05 11:12:00,"3RD DEGREE HEART BLOCK, AORTIC VEGETATION",1,1


In [39]:
patients = pd.read_csv('/Users/grace/mimic/csv/PATIENTS.csv', sep=',')
# patients.head()

In [40]:
patients[patients.SUBJECT_ID==25030]

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
32915,23624,25030,F,2078-04-17 00:00:00,2117-09-05 00:00:00,2117-09-05 00:00:00,2117-09-05 00:00:00,1
